In [198]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# 1. Demand data

In [199]:
# 1.1. Import demand data

demand = pd.read_excel(io='file_excel.xlsx', sheet_name = 'production_order')
demand = demand.sort_values(by=['production_order', 'item_code'], ascending=[True, True])
demand.head()

,production_order,item_code,demand
0,O1,A57,45478
1,O2,A57,16974
2,O3,A57,98636
3,O5,A57,41733


In [200]:
# 1.2. Create lists of production orders and items

po_list= demand['production_order'].unique().tolist()
item_list_demand = demand['item_code'].unique().tolist()
print(f'''List of production orders: {po_list}''')
print(f'''List of items: {item_list_demand}''')

List of production orders: ['O1', 'O2', 'O3', 'O5']
List of items: ['A57']


In [201]:
# 1.3. Combination of production order and item lists

po_item = []
for po in po_list:
    for item in item_list_demand:
        po_item.append((po, item))

df_po_item = pd.DataFrame(po_item, columns=['production_order', 'item_code'])
df_po_item.head()

,production_order,item_code
0,O1,A57
1,O2,A57
2,O3,A57
3,O5,A57


In [202]:
# 1.4. Merge with demand data

df_po_item = df_po_item.merge(demand, on=['production_order', 'item_code'], how='left').replace(np.nan, 0)
df_po_item['demand'] = df_po_item['demand'].astype(int)
df_po_item

,production_order,item_code,demand
0,O1,A57,45478
1,O2,A57,16974
2,O3,A57,98636
3,O5,A57,41733


# 2. Supply data

In [203]:
# 2.1. Import supply data

supply = pd.read_excel(io='file_excel.xlsx', sheet_name = 'inventory')
supply = supply.sort_values(by=['item_code', 'stock_on_hand'], ascending=[True, False])
supply.head()

,bin_no,item_code,stock_on_hand
10,11,A1,9712
7,8,A1,9657
20,21,A1,9297
3,4,A1,9048
19,20,A1,7087


In [204]:
# 2.2. Create lists of bins and items

bin_list= supply['bin_no'].unique().tolist()
item_list_supply = supply['item_code'].unique().tolist()
print(f'''List of bins: {bin_list}''')
print(f'''List of items: {item_list_supply}''')

List of bins: [11, 8, 21, 4, 20, 24, 25, 2, 6, 28, 9, 16, 3, 1, 19, 13, 23, 12, 14, 10, 15, 17, 26, 18, 5, 7, 22, 27, 49, 55, 41, 29, 45, 50, 43, 40, 39, 56, 46, 48, 35, 32, 33, 31, 52, 37, 47, 54, 44, 36, 53, 38, 42, 51, 34, 30, 84, 71, 63, 69, 61, 67, 64, 68, 79, 80, 77, 75, 70, 59, 60, 74, 58, 76, 83, 62, 65, 72, 57, 73, 81, 78, 66, 82, 103, 104, 107, 91, 110, 109, 112, 95, 105, 87, 85, 93, 111, 108, 89, 94, 90, 88, 100, 98, 106, 96, 102, 97, 86, 99, 92, 101, 116, 134, 128, 114, 127, 122, 136, 139, 118, 117, 126, 113, 129, 123, 115, 140, 125, 120, 124, 131, 138, 135, 133, 130, 137, 119, 121, 132, 148, 160, 142, 161, 149, 151, 155, 154, 152, 167, 165, 162, 163, 147, 157, 164, 150, 143, 158, 168, 166, 159, 146, 141, 145, 156, 144, 153, 183, 189, 176, 193, 185, 191, 174, 190, 171, 173, 192, 170, 182, 169, 175, 194, 179, 196, 178, 186, 172, 195, 184, 181, 180, 188, 177, 187, 222, 216, 203, 224, 218, 223, 219, 197, 206, 213, 207, 214, 210, 204, 209, 220, 199, 208, 212, 221, 202, 217, 200

In [205]:
# 2.3. Combination of production order and item lists

bin_item = []
for bin_no in bin_list:
    for item in item_list_supply:
        bin_item.append((bin_no, item))

df_bin_item = pd.DataFrame(bin_item, columns=['bin_no', 'item_code'])
df_bin_item.head()

,bin_no,item_code
0,11,A1
1,11,A10
2,11,A11
3,11,A12
4,11,A13


In [206]:
# 2.4. Merge with supply data

df_bin_item = df_bin_item.merge(supply, on=['bin_no', 'item_code'], how='left').replace(np.nan, 0)
df_bin_item['stock_on_hand'] = df_bin_item['stock_on_hand'].astype(int)
df_bin_item['bin_no'] = df_bin_item['bin_no'].astype(str)
df_bin_item

,bin_no,item_code,stock_on_hand
0,11,A1,9712
1,11,A10,0
2,11,A11,0
3,11,A12,0
4,11,A13,0
...,...,...,...
114683,1766,A63,0
114684,1766,A64,0
114685,1766,A7,0
114686,1766,A8,0


# 3. Optimization model

In [207]:
# 3.1. Sets

I = df_po_item['production_order'].unique()  # set of production orders
J = df_po_item['item_code'].unique()  # set of item codes
K = df_bin_item['bin_no'].unique()  # set of bins
D = df_po_item.set_index(['production_order', 'item_code'])['demand'].to_dict()  # set of demand
S = df_bin_item.set_index(['item_code', 'bin_no'])['stock_on_hand'].to_dict()  # set of supply

In [208]:
# 3.2. Optimization model

# Model
model = gp.Model('bin_allocation')

# Variables
x = model.addVars(I, J, K, vtype=GRB.BINARY, name='yes/no decision whether to allocate item j from bin k to production order i')
y = model.addVars(I, J, K, vtype=GRB.CONTINUOUS, name='the amount of item j from bin k to be allocated to production order i')

# Objective function: Minimize the number of bin allocated to production orders
model.setObjective(gp.quicksum(x[i, j, k] for i in I for j in J for k in K), GRB.MINIMIZE)

# Demand contraint
# For each production order i and item j, the total amount allocated from all the bin must be greater than or equal to the demand
for i in I:
    for j in J:
        model.addConstr(gp.quicksum(y[i, j, k] for k in K) == D[i, j])

# Supply constraint
# For each item j and bin k, the total amount to be allocated to all the production orders must be less than or equal to the stock on hand
for j in J:
    for k in K:
        model.addConstr(gp.quicksum(y[i, j, k] for i in I) <= S[j, k])
        
# Linking constraint
# For each production order i, item j and bin k, if any amount of item j in bin k is allocated to production order i, the corresponding decision variable x[i, j, k] must be equal to 1
for i in I:
    for j in J:
        for k in K:
            model.addConstr(y[i, j, k] - S[j, k] * x[i, j, k] <= 0)

model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (26100.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 8964 rows, 14336 columns and 21616 nonzeros
Model fingerprint: 0x85e31413
Variable types: 7168 continuous, 7168 integer (7168 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+03, 1e+05]
Presolve removed 8820 rows and 14112 columns
Presolve time: 0.00s
Presolved: 144 rows, 224 columns, 448 nonzeros
Variable types: 112 continuous, 112 integer (112 binary)
Found heuristic solution: objective 29.0000000

Root relaxation: objective 2.321564e+01, 211 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbe

# 4. Post-optimization processing

In [209]:
# 4.1. Export the result

# Create result list to store allocation results
results_list = []

for i in I:
    for j in J:
        for k in K:
            if x[i, j, k].X > 0:
                results_list.append({
                    'production_order': i,
                    'item_code': j,
                    'bin_no': k,
                    'amount_allocated': y[i, j, k].X
                })

# Create dataframe
allocation_results = pd.DataFrame(results_list)

# Change data type of columns
allocation_results['bin_no'] = allocation_results['bin_no'].astype(str)
allocation_results['amount_allocated'] = allocation_results['amount_allocated'].astype(int)
allocation_results

,production_order,item_code,bin_no,amount_allocated
0,O1,A57,1472,12348
1,O1,A57,1467,9524
2,O1,A57,1465,8771
3,O1,A57,1481,5315
4,O1,A57,1471,4879
5,O1,A57,1478,4641
6,O2,A57,1464,11033
7,O2,A57,1482,5941
8,O3,A57,1458,16871
9,O3,A57,1459,16055


In [210]:
# 4.2. Merge with supply data to check combine bin decision

allocation_results = allocation_results.merge(df_bin_item, on=['bin_no', 'item_code'], how='left')
allocation_results['gap'] = allocation_results['stock_on_hand'] - allocation_results['amount_allocated']
allocation_results['combine_bin_decision'] = np.where(allocation_results['gap'] > 0, 1, 0)
allocation_results

,production_order,item_code,bin_no,amount_allocated,stock_on_hand,gap,combine_bin_decision
0,O1,A57,1472,12348,12348,0,0
1,O1,A57,1467,9524,9524,0,0
2,O1,A57,1465,8771,8771,0,0
3,O1,A57,1481,5315,5916,601,1
4,O1,A57,1471,4879,4879,0,0
5,O1,A57,1478,4641,4641,0,0
6,O2,A57,1464,11033,11033,0,0
7,O2,A57,1482,5941,8045,2104,1
8,O3,A57,1458,16871,16871,0,0
9,O3,A57,1459,16055,16055,0,0


In [211]:
# 4.3. Export to excel

allocation_results.to_excel('allocation_results.xlsx', index=False)